# Uitwerkingen opgaven data-analyse

## Opgave 1.3

We maken hier gebruik van de error function, gedefinieerd in `scipy.special.erf`. Er is ook een complementaire error function, gedefinieerd als $\mathrm{erfc} = 1 - \mathrm{erf}$. Vergeet niet de factor $\sqrt{2}$, aangezien $\mathrm{erf}$ een vereenvoudigde functie is, en het argument nog omgerekend moet worden.

De waarschijnlijkheid dat een waarde méér dan $1.23\sigma$ van het gemiddelde afligt is gegeven door:

In [ ]:
from math import sqrt
from scipy.special import erf, erfc

In [ ]:
erfc(1.23 / sqrt(2))

De waarschijnlijkheid dat een waarde méér dan $2.43\sigma$ *boven* het gemiddelde ligt is gegeven door:

In [ ]:
erfc(2.43 / sqrt(2)) / 2

De waarschijnlijkheid dat een waarde meer dan $0.5\sigma$, maar minder dan $1.5\sigma$ van het gemiddelde verwijderd is, wordt gegeven door:

In [ ]:
erf(1.5 / sqrt(2)) - erf(.5 / sqrt(2))

We willen weten binnen hoeveel standaarddeviaties van het gemiddelde de waarschijnlijkheid dat een waarde gevonden wordt gelijk is aan $50\,\%$. Gebruik hiervoor de inverse error function `erfinv`. Vergeet de factor $\sqrt{2}$ niet!

In [ ]:
from scipy.special import erfinv

In [ ]:
erfinv(.5) * sqrt(2)

## Opgave 1.4

We maken gebruik van pandas, een python data-analyse pakket. We importeren de datafile:

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('10-metingen.txt')

In [ ]:
print(data)

...en bekijken wat statistieken:

In [ ]:
data.describe()

De standaarddeviatie van het gemiddelde is:

In [ ]:
x = data.x
x.std() / sqrt(x.count())

Het resultaat van ons experiment is dus $x = 71.8 \pm 1.7$ of $x = 71.8(17)$.

Voor De kans op een volgende meting met $x\geq 75$ berekenen we eerst hoeveel standaarddeviaties de meting verwijderd is van het gemiddelde. Vervolgens berekenen we de kans op een meting hóger dan dat aantal standaarddeviaties:

In [ ]:
dist = 75 - x.mean()
dist_sigma = dist / x.std()
erfc(dist_sigma / sqrt(2)) / 2

## Opgave 1.5

We gaan plotten, en we moeten `matplotlib` vertellen dat we de plots in de notebook willen zien:

In [ ]:
%matplotlib inline

We openen de datafile en maken een histogram van $y$:

In [ ]:
data = pd.read_csv('80-metingen.txt')
data.describe()

In [ ]:
data.y.hist()

Het is best interessant dat dit histogram er heel anders uitziet dan het antwoord uit de uitwerkingen. Dat histogram is gemaakt met Origin, en dat lijkt standaard iets beter op zoek te gaan naar aardige bingrenzen. Niet simpelweg het minimum en het maximum nemen en opdelen, maar afronden op mooie, ronde getallen. Handmatig kan dat in Python. Voor stapgroottes kun je het beste `arange` gebruiken. Let er daarbij wel op dat `arange` niet inclusief is. Het maximum wordt niet meegenomen. Als je dat ietsje groter maakt dan de bingrens, dus 101 i.p.v. 100, dan komt het goed:

In [ ]:
import numpy as np
bins = np.arange(20, 101, 10)
data.y.hist(bins=bins)

### Methode A: fitten aan volledige dataset

Het fitten van een bekende distributie (zie `scipy.stats`) aan de data gaat vrij eenvoudig. Daar heb je niet eens een histogram voor nodig:

In [ ]:
from scipy import stats
mean, sigma = stats.norm.fit(data.y)
mean, sigma

De meest waarschijnlijke distributie heeft een gemiddelde van 61.6 en een standaarddeviatie van 16.7. Je krijgt alleen de distributie, genormeerd, dus als je dat samen met het histogram wilt plotten moet je eerst nog schalen met een factor $N^2 / N_\mathrm{bins}$:

In [ ]:
import matplotlib.pyplot as plt

data.y.hist(bins=bins)

scale = data.y.count() ** 2 / len(bins - 1)
x = np.linspace(20, 100, 50)
plt.plot(x, scale * stats.norm.pdf(x, mean, sigma))

Het is dan wellicht interessanter om het histogram weer te geven als waardes met foutenvlaggen. Eerst wil je de waardes van het histogram, zonder te plotten. Dan moet je de middens van de bins berekenen, en dat plotten met de fouten $\sqrt{N}$:

In [ ]:
n, _ = np.histogram(data.y, bins=bins)
xbins = (bins[:-1] + bins[1:]) / 2
plt.errorbar(xbins, n, yerr=np.sqrt(n), fmt='o')

plt.plot(x, scale * stats.norm.pdf(x, mean, sigma))

Merk op dat de waardes van de fit iets verschillen van die van Origin, helaas. De volledige code om het laatste plaatje te maken wordt dan:

In [ ]:
# imports
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# read datafile
data = pd.read_csv('80-metingen.txt')

# calculate histogram
bins = np.arange(20, 101, 10)
n, _ = np.histogram(data.y, bins=bins)

# plot histogram with bin centers and error bars
xbins = (bins[:-1] + bins[1:]) / 2
plt.errorbar(xbins, n, yerr=np.sqrt(n), fmt='o')

# fit normal distribution to data
mean, sigma = stats.norm.fit(data.y)

# plot the scaled distribution
scale = data.y.count() ** 2 / len(bins - 1)
x = np.linspace(20, 100, 50)
plt.plot(x, scale * stats.norm.pdf(x, mean, sigma))

We vertrouwen de data wel. De fit gaat door of vlak langs alle foutenvlaggen.

Dit is wel meer werk dan in Origin, vooral als je nog niet zo handig bent. Zeker ook meer om te onthouden. **Merk op dat de fit gedaan is op de volledige set waarnemingen, dus niet op het histogram. Als er gefit moet worden aan een histogram, of als je een zelf-gedefinieerde functie wilt fitten, dan moet je overstappen naar least-squares fitting.**

### Methode B: fitten aan een histogram

Als je least-squares wilt fitten, gebruik dan de `lmfit` bibliotheek. Die is handiger dan `scipy` gebruiken.

In [ ]:
from lmfit import models

Je kunt veel verschillende modellen gebruiken, b.v. een `GaussianModel`. De werkwijze is als volgt: éérst initialiseer je het model en vervolgens voer je de fit uit. Het is handig om voor de fit een *first guess* op te geven, een afschatting van de parameters. Je kunt dit handmatig doen, maar wij gebruiken hier de method `guess` van het model. Dit geeft een afschatting van de parameters...

In [ ]:
gauss = models.GaussianModel()
first_guess = gauss.guess(n, x=xbins)
first_guess.pretty_print()

...die je vervolgens in de fit kunt stoppen.

In [ ]:
fit = gauss.fit(n, x=xbins, weights=1 / np.sqrt(n), params=first_guess)
print(fit.fit_report())

In [ ]:
fit.plot(numpoints=50)

### Vergelijking van de methodes A en B

Je kunt je nu afvragen welke methode beter is. Bovenstaande fit ziet er prachtig uit. Toch is er nog een subtiliteit: het histogram hangt nogal af van de keuze voor de bins. In bovenstaand voorbeeld hadden we de bins zelf gekozen. Een andere keuze voor de bins, levert een andere fit op:

In [ ]:
n, bins = np.histogram(data.y)
xbins = (bins[:-1] + bins[1:]) / 2

first_guess = gauss.guess(n, x=xbins)
fit = gauss.fit(n, x=xbins, weights=1 / np.sqrt(n), params=first_guess)
print(fit.fit_report())
fit.plot(numpoints=50)

Deze fit heeft een iets ander centrum, een andere breedte, en een fors lagere amplitude.